In [141]:
import numpy as np
import pandas as pd
import geopandas as gpd

## Load sites data, and clean data

In [142]:
sites_df = gpd.read_file(
    './data/raw_data/housing_sites/xn--Bay_Area_Housing_Opportunity_Sites_Inventory__20072023_-it38a.shp'
)

In [143]:
la_sites = sites_df.query('jurisdict == "Los Altos" and rhnacyc == "RHNA5"').copy()
la_sites.fillna(value=np.nan, inplace=True)
la_sites['allowden'] = la_sites['allowden'].astype(float)
la_sites['relcapcty'] = la_sites['relcapcty'].astype(float)

In [144]:
is_constant = ((la_sites == la_sites.iloc[0]).all())
constant_cols = is_constant[is_constant].index.values
constant_cols

array(['rhnacyc', 'rhnayrs', 'county', 'jurisdict', 'allowlow', 'num_vl',
       'num_l', 'num_m', 'num_am', 'infcapcty'], dtype=object)

In [145]:
la_sites.drop(constant_cols, axis=1, inplace=True)

In [146]:
la_sites.dropna(how='all', axis=1, inplace=True)

In [147]:
la_sites.shape

(57, 19)

## Load building permits data

In [148]:
permits = pd.read_csv('./data/clean_data/los_altos_clean.csv', sep='\t')

In [149]:
permits.shape

(249, 15)

In [150]:
permits.head()

,Permit.Number,Permit.Type,Permit.Subtype,Status,Description,Site.Address,APN,Applied,Issued,Finaled,Address,RHNA.Date,Applied.Year,lon,lat
1,1996-630232,MULTI FAMILY,CONDOMINIUM,FINALED,Condo,540 GABILAN ST,17038093.0,1996-08-01,1996-10-31,1997-08-21,540 GABILAN,1996.0,1996,-122.110832,37.373316
2,2006-648097,MULTI FAMILY,NaN,FINALED,Town Homes,4388 EL CAMINO REAL,16711062.0,2006-04-14,2007-09-25,2009-06-04,4388 EL CAMINO,2007.0,2006,-122.117649,37.403486
3,2007-649844,MIXED USE,NaN,PENDING,Test,0 N AVALON DR,17022013.0,2007-04-09,NaN,NaN,0 AVALON DR,NaN,2007,-122.104574,37.385009
4,2008-654927,MULTI FAMILY,NaN,FINALED,Townhouse,58 CUESTA DR,17048024.0,2008-10-13,2010-03-05,2010-11-02,58 CUESTA DR,2010.0,2008,-122.113517,37.373805
5,2008-654928,MULTI FAMILY,NaN,FINALED,Townhome,62 CUESTA DR,17048025.0,2008-10-13,2010-03-05,2010-10-29,62 CUESTA DR,2010.0,2008,-122.113470,37.373808


In [151]:
permits.dropna(how='all', axis='columns', inplace=True)


In [152]:
permits.shape

(249, 15)

In [153]:
date_cols = ['Issued', 'Finaled', 'Applied']
permits[date_cols] = permits[date_cols].apply(pd.to_datetime)

In [154]:
permits.groupby(pd.Grouper(key='Issued', freq='10Y')).size()

Issued
1996-12-31      1
2006-12-31      0
2016-12-31    215
2026-12-31     24
dtype: int64

In [155]:
permits.iloc[1]

Permit.Number             2006-648097
Permit.Type              MULTI FAMILY
Permit.Subtype                    NaN
Status                        FINALED
Description                Town Homes
Site.Address      4388 EL CAMINO REAL
APN                       1.67111e+07
Applied           2006-04-14 00:00:00
Issued            2007-09-25 00:00:00
Finaled           2009-06-04 00:00:00
Address                4388 EL CAMINO
RHNA.Date                        2007
Applied.Year                     2006
lon                          -122.118
lat                           37.4035
Name: 2, dtype: object

In [156]:
permits_15_to_19 = permits.query('2015 <= Issued <= 2020')

In [163]:
la_sites.apn = la_sites.apn.str.replace('-','').astype('float')

In [164]:
la_sites.apn.isin(permits_15_to_19.APN).mean()

0.017543859649122806

In [165]:
la_sites[la_sites.apn.isin(permits_15_to_19.APN)]

,objectid,apn,locapn,genplan,zoning,gacres,locacres,currunits,allowden,allowhigh,relcapcty,siteconst,sitetype,pdaparcel,existuse,localnote,Shape__Are,Shape__Len,geometry
39880,82014,17038047.0,170-38-047,Medium Density Multi-Family,R3-1.8,0.162,0.16,1,50.0,50,8.0,One SFR,Underutilized,NaN,One SFR,could accommodate additional units through inc...,6.664641e-08,0.001253,"POLYGON ((-122.11248 37.37373, -122.11248 37.3..."
